In [22]:
# Import libraries.
import json
from pathlib import Path
from collections import namedtuple

from dataclasses import replace
import pandas as pd
import phoenix as px

In [31]:
import numpy as np
PATH_TRAIN="/data/bacteria/experiments/autoencoders/6mer/26122023-2"
KMER=6
feature_pred_label = "consensus_10"
xq = np.load(f"{PATH_TRAIN}/faiss-embeddings/query_embeddings.npy").astype("float32")

In [24]:
df_labels = pd.read_csv(f"{PATH_TRAIN}/test/test_index.tsv",sep="\t", usecols=["GT",feature_pred_label])

In [25]:
Phoenix=namedtuple("Phoenix",["id","path_img","link_img","label","vector_img"])
data_phoenix = []

with open(f"{PATH_TRAIN}/faiss-embeddings/query_embeddings.json") as fp:
    id_query = json.load(fp)
    id_query = {int(k): v for k,v in id_query.items()}

REPLACE="/data/bacteria/experiments/autoencoders"
for idx, path_npy in id_query.items():
    _id = Path(path_npy).stem
    path_img = path_npy.replace("fcgr","img").replace(".npy",".jpeg")
    link_img = path_img.replace(REPLACE,"http://localhost:6007") # I choose the port 6007, and I need to run `python -m http.server 6007` in the REPLACE directory
    label = Path(path_img).parent.stem.split("__")[0]
    embedding = xq[idx]
    data_phoenix.append(
        Phoenix(_id, path_img, link_img, label, embedding)
    )
    

In [26]:
train_df = pd.concat([pd.DataFrame(data_phoenix), df_labels],axis=1)
train_df.head()

,id,path_img,link_img,label,vector_img,GT,consensus_10
0,SAMN08887978,/data/bacteria/experiments/autoencoders/6mer/i...,http://localhost:6007/6mer/img/brucella_melite...,brucella_melitensis,"[97.584114, 0.0, 0.0, 0.0, 0.0, 522.3128, 0.0,...",brucella_melitensis,brucella_melitensis
1,SAMN03222920,/data/bacteria/experiments/autoencoders/6mer/i...,http://localhost:6007/6mer/img/acinetobacter_b...,acinetobacter_baumannii,"[843.95386, 723.7399, 0.0, 0.0, 0.0, 0.0, 0.0,...",acinetobacter_baumannii,acinetobacter_baumannii
2,SAMEA3357335,/data/bacteria/experiments/autoencoders/6mer/i...,http://localhost:6007/6mer/img/klebsiella_vari...,klebsiella_variicola,"[799.2456, 286.15045, 0.0, 269.867, 0.0, 0.0, ...",klebsiella_variicola,klebsiella_variicola
3,SAMEA29273668,/data/bacteria/experiments/autoencoders/6mer/i...,http://localhost:6007/6mer/img/klebsiella_pneu...,klebsiella_pneumoniae,"[800.6974, 350.22266, 0.0, 221.9124, 0.0, 0.0,...",klebsiella_pneumoniae,klebsiella_pneumoniae
4,SAMEA3403205,/data/bacteria/experiments/autoencoders/6mer/i...,http://localhost:6007/6mer/img/campylobacter_f...,campylobacter_fetus,"[0.0, 1000.60443, 0.0, 0.0, 0.0, 785.8938, 0.0...",campylobacter_fetus,campylobacter_fetus


In [27]:
# Define schemas that tell Phoenix which columns of your DataFrames correspond to features, predictions, actuals (i.e., ground truth), embeddings, etc.
train_schema = px.Schema(
    prediction_id_column_name="id",
    # timestamp_column_name="prediction_ts",
    prediction_label_column_name=feature_pred_label,
    actual_label_column_name="GT",
    embedding_feature_column_names={
        "image_embedding": px.EmbeddingColumnNames(
            vector_column_name="vector_img",
            link_to_data_column_name="link_img",
        ),
    },
)

In [28]:
# Define schemas that tell Phoenix which columns of your DataFrames correspond to features, predictions, actuals (i.e., ground truth), embeddings, etc.
# prod_schema = replace(train_schema, actual_label_column_name=None)

# Define your production and training datasets.
# prod_ds = px.Dataset(prod_df, prod_schema)
train_ds = px.Dataset(train_df, train_schema)

# Launch Phoenix.
session = px.launch_app(
    # prod_ds, 
    train_ds,
    port=6006
    )

# View the Phoenix UI in the browser
# session.url
px.active_session().view()

Existing running Phoenix instance detected! Shutting it down and starting a new instance...


🌍 To view the Phoenix app in your browser, visit http://127.0.0.1:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix
📺 Opening a view to the Phoenix app. The app is running at http://127.0.0.1:6006/


/home/avila/micromamba/envs/phoenix/lib/python3.11/site-packages/phoenix/server/api/types/EmbeddingDimension.py:422: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  prediction_id=dataset[PREDICTION_ID][row_id],
/home/avila/micromamba/envs/phoenix/lib/python3.11/site-packages/phoenix/server/api/types/EmbeddingDimension.py:423: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  link_to_data=dataset[self.dimension.link_to_data][row_id],
/home/avila/micromamba/envs/phoenix/lib/python3.11/site-packages/phoenix/server/api/types/EmbeddingDimension.py:424: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. 